In [1]:
import numpy as np
from os import listdir, mkdir, path
import matplotlib.pyplot as plt
from matplotlib import colors, cm
%matplotlib

Using matplotlib backend: TkAgg


In [2]:
folder1 = '/Users/Amanda/Desktop/PtSn/PtSn1_subtract_background'
bgfile1 = '/Users/Amanda/Desktop/PtSn/PtSn1_subtract_background/background.dat'
loc1 = path.join(folder1, 'addtnl', 'PtSn1')

bounds1 = ([0, 2.69, 0.025, 0, 2.80, 0.025, 0, 2.96, 0.025, 0, 3.12, 0.025],
           [20, 2.79, 0.08, 20, 2.9, 0.08, 20, 3.06, 0.08, 20, 3.23, 0.08])
sequencestarters1 = [path.join(folder1,'_PtSn1_scan1_0031_C.dat'),
                    path.join(folder1,'dSn1_1_scan1_0000_C.dat')]
initialvalues1 = [[10.3, 2.74, 0.05, 1, 2.85, 0.05, 1, 3.01, 0.05, 1.8, 3.17, 0.05],
                 [1, 2.74, 0.05, 5.08, 2.85, 0.05, 2.4, 3.01, 0.05, 1, 3.17, 0.05],
                 [0.7, 2.74, 0.05, 5.65, 2.85, 0.05, 3.2, 3.01, 0.05, 0.3, 3.17, 0.05]]


In [3]:
def mk_subfolders(folder):
    if not path.isdir(path.join(folder, 'bg_subbed')):
        mkdir(path.join(folder, 'bg_subbed'))
    if not path.isdir(path.join(folder, 'addtnl')):
        mkdir(path.join(folder, 'addtnl'))
    if not path.isdir(path.join(folder, 'pdf')):
        mkdir(path.join(folder, 'pdf'))
    if not path.isdir(path.join(folder, 'png')):
        mkdir(path.join(folder, 'png'))

def find_files_PtSn(folder):
    files = listdir(folder)
    files2 = [ii for ii in files if ((not ii.__contains__('background')) & ii.__contains__('dat'))]
    files3 = [path.join(folder,  ii) for ii in files2]
    return files2, files3

def readfiles(folder, bgfile):
    mk_subfolders(folder)
    bg = np.loadtxt(bgfile, skiprows=34)
    bg_q, bg_I, _,_ = bg.T
    filenames, fullpaths = find_files_PtSn(folder)
    nfiles = len(fullpaths)
    intensity_image = np.zeros((bg_q.size,nfiles),dtype=float)
    error_image = np.zeros((bg_q.size,nfiles),dtype=float)
    for ii in range(nfiles):
        spectrum = np.loadtxt(fullpaths[ii], skiprows=34)
        spec_q, spec_I, spec_dI, _ = spectrum.T
        if (spec_q != bg_q).any():
            raise ValueError("Issue!  domain mismatch %s" % fullpaths[ii])
        intensity_image[:,ii] = spec_I
        error_image[:,ii] = spec_dI
    #waterfall_plot(diff_image, bg, loc)
    return intensity_image, error_image, bg, filenames, fullpaths

In [4]:
intensity_image1, error_image1, bg1, names1, files1 = readfiles(folder1, bgfile1)

In [5]:
def waterfall_plot(diff_image, bg, loc):
    nq, nfiles = diff_image.shape
    qq,_,_,_ = bg.T
    viridis = plt.get_cmap('viridis')
    cNorm = colors.Normalize(vmin=0, vmax=nfiles)
    scalarMap = cm.ScalarMappable(norm=cNorm, cmap=viridis)
    fig, ax = plt.subplots()
    ax.plot([qq[0],qq[-1]], [0,0], color='k',lw=1.5)
    for ii in range(nfiles):
        colorVal = scalarMap.to_rgba(ii)
        ax.plot(qq, diff_image[:,ii], color=colorVal, lw=0.5)
    ax.plot([2.3,2.5],[-2,-2],c='r',ls=':')
    ax.plot([3.35,4.1],[-2,-2],c='r',ls=':')
    #ax.plot([4.4,qq[-1]],[-2,-2],c='r',ls=':')
    ax.set_title('Waterfall; background regions')
    fig.savefig(loc + '_waterfall.pdf')
    #plt.close(fig)

In [6]:
waterfall_plot(intensity_image1, bg1, loc1)

In [7]:
def bg_subtract_fit_individual2(intensity_image, filenames, folder, bg, loc, demonstrative=False):
    bg_q, bg_I, _,_ = bg.T
    nfiles = len(filenames)
    
    features = ~(((bg_q > 2.3) & (bg_q < 2.5)) | ((bg_q > 3.35) & (bg_q < 4.1))) # | (bg_q > 4.4))
    diff_image = np.zeros((bg_q.size,nfiles),dtype=float)
    for ii in range(nfiles):
        spec_I = intensity_image[:,ii]
        mult = ((spec_I/bg_I)[~features]).mean()
        diff = spec_I - mult*bg_I
        diff_image[:,ii] = diff
    # Waterfall plot of diffed spectra
    waterfall_plot(diff_image, bg, loc)
    # Image plot of diffed spectra
    fig6, ax6 = plt.subplots()
    ax6.imshow(diff_image.T)
    ax6.set_xlabel('q index')
    ax6.set_ylabel('t index')
    ax6.set_title('Background subtracted spectrum evolution')
    fname = loc+'_diffimage.pdf'
    fig6.savefig(fname)
    #plt.close(fig6)
    return diff_image, features

In [8]:
diff_image1, features1 = bg_subtract_fit_individual2(intensity_image1, files1, folder1, bg1, loc1)

In [26]:
def write_subbed(diff_image,bg,filenames):
    folder = path.join(folder1,'bg_subbed')
    q,_,_,_=bg.T
    X = np.zeros((q.size,2),dtype=float)
    X[:,0] = q
    for ii in range(len(filenames)):
        fname = path.join(folder,filenames[ii])
        X[:,1] = diff_image[:,ii]
        np.savetxt(fname,X,'%.8g','\t',header="q\t\tI_subtracted")

In [27]:
write_subbed(diff_image1,bg1,names1)